In [ ]:
# Zielstellung des Dokuments: Darstellung einer Weltkarte mit eingezeichneten Kreisen, deren Radius proportional ist den Durchschnittstemperaturen der Städte

In [2]:
# Importieren die notwendigen Bibliotheken
import pandas as pd
import folium


C:\Users\torsten.schmidt\AppData\Local\Temp\ipykernel_21856\1943882533.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Einlesen der csv-datei
df = pd.read_csv('globallandtemperaturesbymajorcity.csv')
df

,dt,averagetemperature,averagetemperatureuncertainty,city,country,latitude,longitude
0,1849-01-01,26.704,1.435,Abidjan,Côte D'Ivoire,5.63N,3.23W
1,1849-02-01,27.434,1.362,Abidjan,Côte D'Ivoire,5.63N,3.23W
2,1849-03-01,28.101,1.612,Abidjan,Côte D'Ivoire,5.63N,3.23W
3,1849-04-01,26.140,1.387,Abidjan,Côte D'Ivoire,5.63N,3.23W
4,1849-05-01,25.427,1.200,Abidjan,Côte D'Ivoire,5.63N,3.23W
...,...,...,...,...,...,...,...
239172,2013-05-01,18.979,0.807,Xian,China,34.56N,108.97E
239173,2013-06-01,23.522,0.647,Xian,China,34.56N,108.97E
239174,2013-07-01,25.251,1.042,Xian,China,34.56N,108.97E
239175,2013-08-01,24.528,0.840,Xian,China,34.56N,108.97E


In [4]:
# Umformatierung der Längen und Breitengrade
def konvertiere_breitengrad(breitengrad):
    if 'N' in breitengrad:
        return float(breitengrad[:-1])
    else:
        return -float(breitengrad[:-1])

def konvertiere_laengengrad(laengengrad):
    if 'E' in laengengrad:
        return float(laengengrad[:-1])
    else:
        return -float(laengengrad[:-1])

df['latitude'] = df['latitude'].apply(konvertiere_breitengrad)
df['longitude'] = df['longitude'].apply(konvertiere_laengengrad)
df

,dt,averagetemperature,averagetemperatureuncertainty,city,country,latitude,longitude
0,1849-01-01,26.704,1.435,Abidjan,Côte D'Ivoire,5.63,-3.23
1,1849-02-01,27.434,1.362,Abidjan,Côte D'Ivoire,5.63,-3.23
2,1849-03-01,28.101,1.612,Abidjan,Côte D'Ivoire,5.63,-3.23
3,1849-04-01,26.140,1.387,Abidjan,Côte D'Ivoire,5.63,-3.23
4,1849-05-01,25.427,1.200,Abidjan,Côte D'Ivoire,5.63,-3.23
...,...,...,...,...,...,...,...
239172,2013-05-01,18.979,0.807,Xian,China,34.56,108.97
239173,2013-06-01,23.522,0.647,Xian,China,34.56,108.97
239174,2013-07-01,25.251,1.042,Xian,China,34.56,108.97
239175,2013-08-01,24.528,0.840,Xian,China,34.56,108.97


In [11]:
# Berechnung der Durchschnittstemperatur pro Stadt über alle Jahre
df_agg = df.groupby(['city', 'latitude', 'longitude'])['averagetemperature'].aggregate('mean').reset_index()
df_agg

,city,latitude,longitude,averagetemperature
0,Abidjan,5.63,-3.23,26.163737
1,Addis Abeba,8.84,38.11,17.525073
2,Ahmadabad,23.31,72.52,26.529853
3,Aleppo,36.17,37.79,17.370587
4,Alexandria,31.35,30.16,20.312617
...,...,...,...,...
95,Tokyo,36.17,139.23,12.555998
96,Toronto,44.20,-80.50,5.773911
97,Umm Durman,15.27,32.50,29.081291
98,Wuhan,29.74,114.46,16.830944


In [12]:
# Zeichen der Hintergrund-Weltkarte in Folium
karte = folium.Map(location=[0,0], zoom_start=2)


In [15]:
# Kreise auf Karte für jede Stadt mit r ~ T_average
for index, row in df_agg.iterrows():
    kreisradius = row['averagetemperature']*10000
    folium.Circle(
        location=(row['latitude'], row['longitude']),
        radius=kreisradius,
        color = 'red',
        fill = True,
        fill_color = 'red',
        fill_opacity =0.6,
        popup = row['city'] +':'+str(round(row['averagetemperature']))+' °C'
    ).add_to(karte)

In [16]:
karte

In [17]:
# Speichere die Karte als html-Datei
karte.save('weltkarte_der_Temperaturen.html')